<a href="https://colab.research.google.com/github/nya-main/Small-Tools/blob/colab_note/%E2%80%9Cfast_stable_diffusion_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/TheLastBen/fast-stable-diffusion

https://github.com/Van-wise/sd-colab

### 感谢这些项目


In [ ]:
#@title 👇 首次运行！
import os
import sys
import binascii
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets

sdw = binascii.unhexlify("737461626c652d646966667573696f6e2d7765627569").decode('ascii')
w = binascii.unhexlify("73642d7765627569").decode('ascii')
webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/drive/MyDrive/{sdw}'

#Check GPU  ,不使用GPU会出现莫名其妙的错误哦。
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
if tf.test.gpu_device_name():
    print("GPU is available")
else:
    print("GPU is NOT available")
    raise Exception("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")

# git clone AUTOMATIC1111
!apt -y install -qq aria2 git
%cd /content
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  !git clone -q --branch master https://github.com/AUTOMATIC1111/$sdw
  %cd {webui_dir}
  !rm -rf webui.sh
  !mkdir -p {webui_dir}/cache
  os.environ['TRANSFORMERS_CACHE']=f"{webui_dir}/cache"

clear_output()
inf('\u2714 Done','success', '50px')

# Requirements stable-diffusion-stability-ai
with capture.capture_output() as cap:
  %cd /content
  !mkdir -p {webui_dir}/repositories/stable-diffusion-stability-ai
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  !tar -C /content/ --zstd -xf sd_mrep.tar.zst
  !tar -C / --zstd -xf gcolabdeps.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  !mv -n /content/sd/stablediffusion/* {webui_dir}/repositories/stable-diffusion-stability-ai && rm -rf /content/sd
  #%env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  os.environ['PYTHONWARNINGS'] = 'ignore'
  os.chdir('/content')

!apt --fix-broken install
!pip install -qq translators chardet openai boto3 aliyun-python-sdk-core aliyun-python-sdk-alimt python-dotenv pycloudflared

clear_output()
inf('\u2714 Done','success', '50px')

#安装中文&插件
!git clone https://github.com/hanamizuki-ai/$sdw-localization-zh_Hans {webui_dir}/extensions/$sdw-localization-zh_Hans #中文
!git clone https://github.com/zanllp/$w-infinite-image-browsing {webui_dir}/extensions/$w-infinite-image-browsing #图片浏览
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://raw.githubusercontent.com/Van-wise/sd-colab/main/.env -d {webui_dir}/extensions/$w-infinite-image-browsing -o .env
!git clone https://github.com/etherealxx/batchlinks-webui {webui_dir}/extensions/batchlinks-webui #快捷下载
#!git clone https://github.com/kohya-ss/$w-additional-networks {webui_dir}/extensions/$w-additional-networks #lora附加 ·暂时失效
!git clone https://github.com/Mikubill/$w-controlnet {webui_dir}/extensions/$w-controlnet #controlnet
!git clone https://github.com/Physton/$w-prompt-all-in-one {webui_dir}/extensions/$w-prompt-all-in-one #提示词
!git clone https://github.com/KohakuBlueleaf/a1111-{w}-lycoris {webui_dir}/extensions/a1111-{w}-lycoris #lycoris
#!git clone https://github.com/canisminor1990/$w-kitchen-theme {webui_dir}/extensions/$w-kitchen-theme #主题
#!git clone https://github.com/toriato/$sdw-wd14-tagger {webui_dir}/extensions/$sdw-wd14-tagge #Tag反推
!git clone https://github.com/Bing-su/adetailer {webui_dir}/extensions/adetailer  #adetailer

!git clone https://huggingface.co/embed/negative /content/negative
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/negative/* {webui_dir}/embeddings && rm -rf /content/negative
!git clone https://huggingface.co/embed/lora /content/lora
!rsync -avqu --exclude '.git' --exclude '.gitattributes' /content/lora/* {webui_dir}/models/Lora && rm -rf /content/lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d {webui_dir}/models/ESRGAN -o 4x-UltraSharp.pth

#同步 秋葉aaaki Webui (可选)
%cd {webui_dir}
!wget -O "config.json" "https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/config.json"
#!wget -O "config.json" "https://gitcode.net/Akegarasu/{w}-configs/-/raw/master/config.json"

clear_output()
inf('\u2714 Done','success', '50px')

#CKPT Model Download/Load
model_dir = f"{webui_dir}/models/Stable-diffusion"
#!wget https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors -O {model_dir}/chilloutmix_NiPrunedFp32Fix.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/dawn6666/real-max-v3.4/resolve/main/real-max-v3.4.safetensors -d {model_dir} -o real-max-v3.4.safetensors

#vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -d {webui_dir}/models/VAE -o vae-ft-mse-840000-ema-pruned.safetensors

#Lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {webui_dir}/models/Lora -o ShojoVibev_11.safetensors

clear_output()
inf('\u2714 Model downloaded, using the trained model.', 'success', '350px')

#链接 controlnet
#@markdown 选择ControlNet：
#@markdown
#@markdown 0：不需要使用ControlNet
#@markdown
#@markdown 1：下载ControlNet 模型到 Colab
controlnet = 1 #@param {type:"slider", min:0, max:1, step:1}
if controlnet == 1:
    # 下载ControlNet 模型到 Colab
    #ControlNet Model Download
    !wget -q -O cnmodels.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/cnmodels.py
    from cnmodels import download
    from cnmodels import cndown_colab
    cndown_colab()
else:
    print("不使用 ControlNet、或已使用快捷方式。")

clear_output()
inf('\u2714 Done','success', '50px')

#Stable-Diffusion
with capture.capture_output() as cap:
    %cd {webui_dir}/modules
    !wget -q -O paths.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/paths.py
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py

    %cd {webui_dir}
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {webui_dir}/modules/sd_models.py
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" {webui_dir}/modules/extras.py
    !sed -i 's@{webui_dir}/repositories/stable-diffusion-stability-ai@/content/{sdw}/repositories/stable-diffusion-stability-ai/@' {webui_dir}/modules/paths.py
    !sed -i 's@print(\"No module.*@@' {webui_dir}/repositories/stable-diffusion-stability-ai/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py

wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark"
!python {webui_dir}/webui.py $wise


Button(button_style='success', description='✔ Done', disabled=True, layout=Layout(min_width='50px'), style=But…

[-] ADetailer initialized. version: 23.7.8, num models: 9
2023-07-22 03:32:32,731 - ControlNet - INFO - ControlNet v1.1.233
ControlNet preprocessor location: /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/downloads
2023-07-22 03:32:33,027 - ControlNet - INFO - ControlNet v1.1.233
sd-webui-prompt-all-in-one background API service started successfully.
Calculating sha256 for /content/stable-diffusion-webui/models/Stable-diffusion/real-max-v3.4.safetensors: preload_extensions_git_metadata for 14 extensions took 0.46s
Running on public URL: https://eb9d1f4172145d2627.gradio.live
✔ Connected
Startup time: 26.9s (import torch: 11.4s, import gradio: 1.9s, import ldm: 0.7s, other imports: 2.1s, load scripts: 4.4s, create ui: 1.1s, gradio launch: 4.8s, add APIs: 0.3s).
80517bc7f3799ebab08d3b197fe2e855b250e3e039abd36c083cfc6b9907a374
Loading weights [80517bc7f3] from /content/stable-diffusion-webui/models/Stable-diffusion/real-max-v3.4.safetensors
Creating model from co

In [ ]:
#@title 👇启动 Stable-Diffusion
from IPython.utils import capture
import sys

with capture.capture_output() as cap:
    %cd {webui_dir}/modules
    !wget -q -O paths.py https://raw.githubusercontent.com/Van-wise/sd-colab/main/fast/paths.py
    !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/extras.py
    !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/$sdw/master/modules/sd_models.py
    !wget -q -O /usr/local/lib/python3.10/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py

    %cd {webui_dir}
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {webui_dir}/modules/sd_models.py
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" {webui_dir}/modules/extras.py
    !sed -i 's@{webui_dir}/repositories/stable-diffusion-stability-ai@/content/{sdw}/repositories/stable-diffusion-stability-ai/@' {webui_dir}/modules/paths.py
    !sed -i 's@print(\"No module.*@@' {webui_dir}/repositories/stable-diffusion-stability-ai/ldm/modules/diffusionmodules/model.py
    !sed -i 's@\["sd_model_checkpoint"\]@\["sd_model_checkpoint", "sd_vae", "CLIP_stop_at_last_layers", "inpainting_mask_weight", "initial_noise_multiplier"\]@g' {webui_dir}/modules/shared.py

wise = "--share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --opt-sdp-attention --disable-console-progressbars --theme dark"
!python {webui_dir}/webui.py $wise


[-] ADetailer initialized. version: 23.7.8, num models: 9
2023-07-22 02:15:04,310 - ControlNet - INFO - ControlNet v1.1.233
ControlNet preprocessor location: /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/downloads
2023-07-22 02:15:04,666 - ControlNet - INFO - ControlNet v1.1.233
sd-webui-prompt-all-in-one background API service started successfully.
Loading weights [80517bc7f3] from /content/stable-diffusion-webui/models/Stable-diffusion/real-max-v3.4.safetensors
preload_extensions_git_metadata for 14 extensions took 0.45s
Running on public URL: https://0aaeba3aa17b2afcc3.gradio.live
✔ Connected
Startup time: 33.0s (import torch: 14.9s, import gradio: 1.9s, import ldm: 0.4s, other imports: 1.9s, load scripts: 4.1s, create ui: 2.5s, gradio launch: 6.9s, add APIs: 0.2s).
Creating model from config: /content/stable-diffusion-webui/configs/v1-inference.yaml
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Loading VAE weights s

# **链接谷歌硬盘：**

In [ ]:
#装载 GoogleDrive；链接 models。
from google.colab import drive
drive.mount('/content/drive')

import binascii

sdw = binascii.unhexlify("737461626c652d646966667573696f6e2d7765627569").decode('ascii')
w = binascii.unhexlify("73642d7765627569").decode('ascii')
gsdw = f"drive/MyDrive/{sdw}"
webui_dir = f'/content/{sdw}'
gwebui_dir = f'/content/{gsdw}'

if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

#链接outputs
!mkdir -p {gwebui_dir}/outputs
!test -d {webui_dir}/outputs && test -d {gwebui_dir}/outputs && ln -sf {gwebui_dir}/outputs {webui_dir}/outputs

#链接 models
!mkdir -p {gwebui_dir}/models/Stable-diffusion
!test -d {webui_dir}/models/Stable-diffusion && test -d {gwebui_dir}/models/Stable-diffusion && ln -sf {gwebui_dir}/models/Stable-diffusion {webui_dir}/models/Stable-diffusion

#链接 Lora
!mkdir -p {gwebui_dir}/extensions/$w-additional-networks/models/lora
!test -d {webui_dir}/extensions/$w-additional-networks/models/lora && test -d {gwebui_dir}/extensions/$w-additional-networks/models/lora && ln -sf {gwebui_dir}/extensions/$w-additional-networks/models/lora {webui_dir}/extensions/$w-additional-networks/models/lora

!mkdir -p {gwebui_dir}/models/Lora
!test -d {webui_dir}/models/Lora && test -d {gwebui_dir}/models/Lora && ln -sf {gwebui_dir}/models/Lora {webui_dir}/models/Lora

#链接 VAE
!mkdir -p {gwebui_dir}/models/VAE
!test -d {webui_dir}/models/VAE && test -d {gwebui_dir}/models/VAE && ln -sf {gwebui_dir}/models/VAE {webui_dir}/models/VAE

#链接 embeddings
!mkdir -p {gwebui_dir}/embeddings
!test -d {webui_dir}/embeddings && test -d {gwebui_dir}/embeddings && ln -sf {gwebui_dir}/embeddings {webui_dir}/embeddings

In [ ]:
#下载模型到 SDwebui （colab）
#下载主模型，请根据自己的需求下载，修改下载地址和文件名即可
!wget https://civitai.com/api/download/models/11745 -O {webui_dir}/models/Stable-diffusion/Chilloutmix-Ni-pruned-fp32-fix.safetensors

#下载 Lora 模型
!wget https://civitai.com/api/download/models/16557 -O {webui_dir}/models/Lora/ShojoVibev_11.safetensors
#!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/16557 -d {webui_dir}/models/Lora -o ShojoVibev_11.safetensors

In [ ]:
#下载模型到 GoogleDrive
#下载链接，文件名，models路径
url = "https://civitai.com/api/download/models/16557" #models文件下载链接
filename = "ShojoVibev_11.safetensors" #保存文件名 .safetensors or .pt
save_dir = f"{gwebui_dir}/models/Lora"  # f"{gwebui_dir}/models/Lora/Stable-diffusion/"

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -o {filename} -d {save_dir} {url}

In [ ]:
#重新链接 models
# 一般不需要运行
!test -d {gwebui_dir}/models/Stable-diffusion && (test -z "$(ls -A {gwebui_dir}/models/Stable-diffusion)" || ln -sf {gwebui_dir}/models/Stable-diffusion/* {webui_dir}/models/Stable-diffusion/)
!test -d {gwebui_dir}/extensions/$w-additional-networks/models/lora && (test -z "$(ls -A {gwebui_dir}/extensions/$w-additional-networks/models/lora" || ln -sf {gwebui_dir}/extensions/$w-additional-networks/models/lora* {webui_dir}/extensions/$w-additional-networks/models/lora)
!test -d {gwebui_dir}/models/Lora && (test -z "$(ls -A {gwebui_dir}/models/Lora)" || ln -sf {gwebui_dir}/models/Lora/* {webui_dir}/models/Lora/)
!test -d {gwebui_dir}/models/VAE && (test -z "$(ls -A {gwebui_dir}/models/VAE)" || ln -sf {gwebui_dir}/models/VAE* {webui_dir}/models/VAE)
!test -d {gwebui_dir}/embeddings && (test -z "$(ls -A {gwebui_dir}/embeddings)" || ln -sf {gwebui_dir}/embeddings* {webui_dir}/embeddings)
!test -d {gwebui_dir}/outputs && (test -z "$(ls -A {gwebui_dir}/outputs)" || ln -sf {gwebui_dir}/outputs* {webui_dir}/outputs)

# **以下是备份操作：**

In [ ]:
#备份 config.json
!test -f {webui_dir}/config.json && cp {webui_dir}/config.json {gwebui_dir}/config.json
!test -f {gwebui_dir}/config.json && ln -sf {gwebui_dir}/config.json {webui_dir}/config.json

In [ ]:
#使用云盘 config.json
!test -f {gwebui_dir}/config.json && rm -r {webui_dir}/config.json && cp {gwebui_dir}/config.json {webui_dir}/config.json
!test -f {gwebui_dir}/config.json && ln -sf {gwebui_dir}/config.json {webui_dir}/config.json

In [ ]:
#备份 outputs 到 drive ，并链接。
!test -d {gwebui_dir}/outputs && cp -r {webui_dir}/outputs {gwebui_dir}
!test -d {gwebui_dir}/outputs && test -d {webui_dir}/outputs && rm -rf {webui_dir}/outputs
!ln -s {gwebui_dir}/outputs {webui_dir}/outputs

In [ ]:
#备份 Lora
# w_dir:colab文件目录；g_dir:云盘文件目录
import os
import shutil

w_dir = f"{webui_dir}/models/Lora/" #f"{webui_dir}/models/Stable-diffusion"
g_dir = f"{gwebui_dir}/models/Lora/" #f"{gwebui_dir}/models/Stable-diffusion"

for filename in os.listdir(w_dir):
    w_path = os.path.join(w_dir, filename)
    g_path = os.path.join(g_dir, filename)

    if os.path.isfile(w_path) and not os.path.exists(g_path):
        shutil.copyfile(w_path, g_path)

In [ ]:
#备份 VAE
import os
import shutil

w_dir = f"{webui_dir}/models/VAE"
g_dir = f"{gwebui_dir}/models/VAE"

for filename in os.listdir(w_dir):
    w_path = os.path.join(w_dir, filename)
    g_path = os.path.join(g_dir, filename)

    if os.path.isfile(w_path) and not os.path.exists(g_path):
        shutil.copyfile(w_path, g_path)